# Audio-Midi Transcription

## Setup and Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import time

import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import torch.utils.data as data_utils


import MidiUtils as mu
import PlotUtils as pu
from Dataset import MidiDataset
from Model import MidiTranscriptionModel
from Note import Note
from Song import Song

dataset_path = "/Users/andreas/Development/Midi-Conversion/maestro-v3.0.0"
workspace = "/Users/andreas/Development/Midi-Conversion/PianoTranscription"

In [ ]:
def get_data_loaders(batch_size=1, time_discretization=100, train_set_size=None, val_set_size=None, shuffle=True):
    # Only create train, validation, test splits if they don't exist (unnecessarily slow)
    if not (os.path.exists(os.path.join(dataset_path, 'train.txt')) and os.path.exists(os.path.join(dataset_path, 'test.txt')) and os.path.exists(os.path.join(dataset_path, 'validation.txt'))):
        MidiDataset.create_dataset_files(dataset_path)

    train_set = MidiDataset(dataset_path, 'train', time_discretization)
    val_set = MidiDataset(dataset_path, 'validation', time_discretization)
    test_set = MidiDataset(dataset_path, 'test', time_discretization)

    if train_set_size is not None:
        limit_type, limit = train_set_size
        if limit_type == 'items':  # At least 1 item, at most all items
            limit = max(min(limit, len(train_set)), 1)
        elif limit_type == 'percentage':  # At least 1 item, at most all items
            limit = max(min(limit * len(train_set), len(train_set)), 1)
        else:
            limit = len(train_set)
        train_set = data_utils.Subset(train_set, torch.arange(limit))

    if val_set_size is not None:
        limit_type, limit = val_set_size
        if limit_type == 'items':  # At least 1 item, at most all items
            limit = max(min(limit, len(val_set)), 1)
        elif limit_type == 'percentage':  # At least 1 item, at most all items
            limit = max(min(limit * len(val_set), len(val_set)), 1)
        else:
            limit = len(val_set)
        val_set = data_utils.Subset(val_set, torch.arange(limit))

    train_data = DataLoader(train_set, batch_size=batch_size, shuffle=shuffle)
    val_data = DataLoader(val_set, batch_size=batch_size, shuffle=shuffle)
    test_data = DataLoader(test_set, batch_size=batch_size, shuffle=shuffle)

    return train_data, val_data, test_data


def create_tqdm_bar(iterable, desc):
    return tqdm(enumerate(iterable), total=len(iterable), ncols=150, desc=desc)

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'

print(device)

## Training

### Overfit on single song

In [ ]:
def train_loop(model: torch.nn.Module, data_loaders, params, workspace, log_name, function):
    assert 'epochs' in params, 'Number of epochs not specified in params (\'epochs\')'
    assert 'device' in params, 'Device not specified in params (\'device\')'
    assert 'batch_size' in params, 'Batch size not specified in params (\'batch_size\')'
    assert 'learning_rate' in params, 'Learning rate not specified in params (\'learning_rate\')'

    model.to(params['device'])

    logger_path = os.path.join(workspace, 'logs', log_name)
    num_of_runs = len(os.listdir(logger_path)) if os.path.exists(logger_path) else 0
    logger = SummaryWriter(os.path.join(logger_path, f'run_{num_of_runs + 1}'))

    epochs = params['epochs']
    train_loader, val_loader, test_loader = data_loaders
    last_time_ms = time.time()/10e6
    val_loss = 0
    for epoch in range(epochs):
        # Create a progress bar for the training loop.
        training_loop = create_tqdm_bar(
            train_loader, desc=f'Training Epoch [{epoch + 1}/{epochs}]')

        train_loss = 0
        for train_iteration, batch in training_loop:
            # Actual training
            loss = model.training_step(batch)
            train_loss += loss

            # Progress indicator
            if time.time_ns()/10e6 - last_time_ms > 10:
                training_loop.set_postfix(curr_train_loss="{:.8f}".format(
                    train_loss / (train_iteration + 1)), val_loss="{:.8f}".format(val_loss), refresh=True)
                last_time_ms = time.time_ns()/10e6
                logger.add_scalar(f'{log_name}/train_loss', loss, epoch * len(train_loader) + train_iteration)

        print(f'Training Epoch [{epoch + 1}/{epochs}] - Train Loss: {train_loss / len(train_loader)}')
        train_loss = train_loss / len(train_loader)

        # Validation
        val_loss = 0
        for val_iteration, batch in enumerate(val_loader):
            # Actual validation
            loss = model.validation_step(batch)
            val_loss += loss
            logger.add_scalar(f'{log_name}/val_loss', loss,
                              epoch * len(val_loader) + val_iteration)

In [ ]:
params = {
    'device': 'cpu',
    'batch_size': 1,
    'train_set_size': ('items', 1), # ('percent', 0.1) = 10 percent, None = all
    'val_set_size': ('items', 1), # ('percent', 0.1) = 10 percent, None = all
    'discretization': 100,

    'learning_rate': 3e-2,
    'epochs': 1,

    'input_size': 480,
    'output_size': 128,
}

loaders = get_data_loaders(params['batch_size'], params['discretization'], train_set_size=params['train_set_size'], val_set_size=params['val_set_size'], shuffle=False)
model = MidiTranscriptionModel(params=params)

train_loop(function=None, model=model, data_loaders=loaders, params=params, workspace=workspace, log_name='overfitting')

## Inference 

In [ ]:
# Load a song
train_loader, _, _ = loaders
audio, midi = train_loader.dataset[0]
pred_midi = model.predict(audio)
pu.plot_tensor_as_image(pred_midi.T)

In [ ]:
Song.start_time_tensor_to_midi(pred_midi, 'predicted_midi.midi', 100)

In [ ]:
# Test back and forth conversion of simple midi
midi = MidiDataset.transform_midi_file('output.mid', 100)
pu.plot_tensor_as_image(midi)
Song.start_time_tensor_to_midi(midi, 'converted_output.mid', 100)